<h2 style="margin:.5em 0;">Einführung in Data Science</h2>
<h3 style="margin:.5em 0;">Fakultät Informatik &ndash; Hochschule Aalen</h3>
<h4 style="margin:.5em 0;">Autoren: Florian Merlau</h4>

#  Bibliotheken importieren 

In [63]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# Datenbanken importieren

In [64]:
raw_ehresp = pd.read_csv('ehresp_2016s.dat')

raw_atusrost = pd.read_csv('atusrost_2016.dat')
raw_atussum = pd.read_csv('atussum_2016.dat')
raw_ehresp.head()

,TUCASEID,TULINENO,EEINCOME1,ERBMI,ERHHCH,ERINCOME,ERSPEMCH,ERTPREAT,ERTSEAT,ETHGT,...,EUMEAT,EUMILK,EUPRPMEL,EUSODA,EUSTORES,EUSTREASON,EUTHERM,EUWGT,EUWIC,EXINCOME1
0,20160101160045,1,2,26.6,3,3,5,40,0,0,...,1,2,1,2,2,4,2,165,-1,0
1,20160101160066,1,2,44.3,3,3,5,30,15,0,...,1,2,1,-1,1,2,2,300,-1,0
2,20160101160069,1,2,24.5,3,5,-1,75,0,0,...,1,2,1,-1,1,5,1,143,2,0
3,20160101160083,1,2,21.2,1,5,-1,165,0,0,...,1,2,1,2,1,1,2,98,2,0
4,20160101160084,1,1,29.7,3,1,3,30,27,0,...,2,2,3,2,1,1,-1,173,-1,0


# Analyse der Daten

In [65]:
[df_male,df_female]
x=len(raw_atusrost)
y=len(raw_atussum)
z=len(raw_ehresp)
[x,y,z]


[27884, 10493, 10210]

## Fehlerhafte Spalten = 423 --> 423 ungültige Einträge entfernen oder mit geeigneten füllen --> siehe Lösung Aufgabenblatt 1

## Gender-Aufteilung zwischen Mann & Frau  "TESEX =1 --> df_male, "TESEX" = 2 -->df_female 
## Gesamte Länge = 10439
## Länge_Mann = 4670, Länge_Frau, 5823


In [66]:
raw_atusrost.columns.values
df_male = raw_atussum[raw_atussum['TESEX'] == 1 ].value_counts().sum() 
df_female = raw_atussum[raw_atussum['TESEX'] == 2].value_counts().sum()


In [67]:


##df_isna =  raw_atussum[raw_atussum['TESEX'] != 1 & 2].value_counts().sum()
df_isna= raw_atussum.isna().sum()
df_isna= df_isna.value_counts().sum()
df_isna


423

## Mergen von Atussum & Ehresp --> aufgrund von fehlendem BMI in atussum 

In [68]:
merged_inner= pd.merge(left=raw_atussum,right=raw_ehresp, left_on='TUCASEID',right_on='TUCASEID')
m

,TUCASEID,TUFINLWGT,TRYHHCHILD,TEAGE,TESEX,PEEDUCA,PTDTRACE,PEHSPNON,GTMETSTA,TELFS,...,EUMEAT,EUMILK,EUPRPMEL,EUSODA,EUSTORES,EUSTREASON,EUTHERM,EUWGT,EUWIC,EXINCOME1
0,20160101160045,2.458865e+07,-1,62,2,39,1,2,1,5,...,1,2,1,2,2,4,2,165,-1,0
1,20160101160066,5.445941e+06,-1,69,1,37,2,2,2,5,...,1,2,1,-1,1,2,2,300,-1,0
2,20160101160069,8.782622e+06,0,24,2,39,2,2,1,5,...,1,2,1,-1,1,5,1,143,2,0
3,20160101160083,3.035910e+06,8,31,2,40,1,2,2,1,...,1,2,1,2,1,1,2,98,2,0
4,20160101160084,6.978586e+06,-1,59,2,39,1,2,1,1,...,2,2,3,2,1,1,-1,173,-1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10205,20161212162307,1.295883e+07,-1,52,2,44,2,2,1,4,...,1,2,1,2,2,1,2,202,-1,0
10206,20161212162357,4.899612e+06,-1,69,1,43,1,2,1,5,...,1,2,1,1,3,1,2,174,-1,0
10207,20161212162426,1.597792e+07,-1,28,1,43,1,1,1,1,...,1,2,1,2,2,2,2,158,-1,0
10208,20161212162456,6.280644e+06,-1,80,2,39,1,2,1,5,...,1,2,1,2,-1,-1,2,270,-1,0


In [69]:
#bank = reduce(
    #lambda left, right: pd.merge(left, right, on=['country', 'year'], how='outer'),
    #data_collection
#)
#bank.head()

merged_inner_male = merged_inner[merged_inner['TESEX'] == 1 ]
merged_inner_female = merged_inner[merged_inner['TESEX'] == 2]
[merged_inner_male,merged_inner_female]





[             TUCASEID     TUFINLWGT  TRYHHCHILD  TEAGE  TESEX  PEEDUCA  \
 1      20160101160066  5.445941e+06          -1     69      1       37   
 8      20160101160139  9.112257e+06          -1     63      1       46   
 10     20160101160546  3.165768e+06           1     35      1       43   
 16     20160101160573  9.150706e+06          16     16      1       36   
 17     20160101160574  1.299119e+06          13     70      1       40   
 ...               ...           ...         ...    ...    ...      ...   
 10199  20161212162015  2.775790e+06          -1     72      1       40   
 10200  20161212162045  1.985660e+07          17     17      1       37   
 10203  20161212162212  1.986985e+07          -1     75      1       39   
 10206  20161212162357  4.899612e+06          -1     69      1       43   
 10207  20161212162426  1.597792e+07          -1     28      1       43   
 
        PTDTRACE  PEHSPNON  GTMETSTA  TELFS  ...  EUMEAT  EUMILK  EUPRPMEL  \
 1             2    

In [74]:
merged_male=merged_inner_male.drop(columns=['ETWGT', 'EUWGT'] )
merged_female=merged_inner_female.drop(columns=['ETWGT', 'EUWGT'])
#raw.drop(columns=['SibSp', 'Parch','PassengerId'], inplace=True)
#raw


## Alle Prädiktionen die mit dem BMI oder Gewicht verbunden sind, wurden gelöscht

In [76]:
merged_male.columns
merged_female.columns

Index(['TUCASEID', 'TUFINLWGT', 'TRYHHCHILD', 'TEAGE', 'TESEX', 'PEEDUCA',
       'PTDTRACE', 'PEHSPNON', 'GTMETSTA', 'TELFS',
       ...
       'EUINCOME2', 'EUMEAT', 'EUMILK', 'EUPRPMEL', 'EUSODA', 'EUSTORES',
       'EUSTREASON', 'EUTHERM', 'EUWIC', 'EXINCOME1'],
      dtype='object', length=457)